In [1]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
with np.load('data/octa_sphere_5.npz') as npl:
    v_s, f_s = npl['v'], npl['f']

filename="woody-hi"
v, f = igl.read_triangle_mesh(f'data/{filename}.off')
segments_recording = np.zeros(len(v), dtype=int)
value=[]

In [2]:
v -= v.min(axis=0)
v /= v.max()

In [3]:
tb = iw.ToggleButtons(description='Segment', options=[1,'+'])
@mp.interact(x=tb)
def s_but(x):
    if x == '+':
        tb.options = list(range(1,len(tb.options)+1)) + ['+']
        tb.value = len(tb.options) - 1
    else:
        tb.current = x

button = iw.Button(description="Paint!")
clear_button = iw.Button(description="Clear!")

# Set Callback
def paint_clicked(b):
    slicer = np.where(np.linalg.norm(v - sf.coord[1:],axis=1) < sf.coord[0])[0]
    value.append(sf.coord[1:])
    segments_recording[slicer] = tb.current
    paint_ui.update_object(oid=0, colors=segments_recording)
    paint_ui.add_points(np.array(sf.coord[1:])[None,:],shading={"point_size": 0.3})
def clear_clicked(b):
    value=[]
    segments_recording[segments_recording == tb.current] = 0
    paint_ui.update_object(oid=0, colors=segments_recording)
button.on_click(paint_clicked)
clear_button.on_click(clear_clicked)

# Display Buttons
display(iw.HBox([button, clear_button]))

# Meshplot
paint_ui = mp.plot(v,f,c=segments_recording)
paint_ui.add_mesh(v_s*0.1, f_s,shading={"flat" : False},c=np.array([1,0,0]))
def sf(radius,x,y,z):
    paint_ui.update_object(oid = 1, vertices = v_s*radius + np.array([x,y,z]))
    sf.coord = [radius,x,y,z]
mp.interact(sf, 
            radius = iw.FloatSlider(min=0.01, max=0.1, value=0.01,step=0.01),
            x = iw.FloatSlider(min=-0.1, max=1, value=0,step=0.01),
            y = iw.FloatSlider(min=-0.1, max=1.5, value=0,step=0.01),
            z = iw.FloatSlider(min=-0.1, max=1, value=0,step=0.01))

interactive(children=(ToggleButtons(description='Segment', options=(1, '+'), value=1), Output()), _dom_classes…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

interactive(children=(FloatSlider(value=0.01, description='radius', max=0.1, min=0.01, step=0.01), FloatSlider…

<function __main__.sf(radius, x, y, z)>

In [6]:
value

[[0.20000000000000004, 0.0, 0.0],
 [0.28, -0.04000000000000001, 0.0],
 [0.39, -0.010000000000000009, 0.0],
 [0.44000000000000006, 0.21, 0.0],
 [0.52, 0.0, 0.0],
 [0.62, -0.03, 0.0],
 [0.74, 0.009999999999999995, 0.0],
 [0.75, 0.16, 0.0],
 [0.65, 0.49, 0.0],
 [0.8900000000000001, 0.49, 0.0],
 [0.8900000000000001, 0.7200000000000002, 0.0],
 [0.62, 0.73, 0.0],
 [0.5800000000000001, 0.9300000000000003, 0.0],
 [0.41000000000000003, 1.0, 0.0],
 [0.22, 0.9400000000000001, 0.0],
 [0.19000000000000003, 0.81, 0.0],
 [0.26, 0.73, 0.0],
 [-0.03, 0.73, 0.0],
 [-0.03, 0.4800000000000001, 0.0],
 [0.23, 0.4800000000000001, 0.0],
 [0.13000000000000003, 0.12, 0.0]]

In [8]:
value[13][1]=1.1
value[13]

[0.41000000000000003, 1.1, 0.0]

In [10]:
temp=np.array(value)
np.save(f'data/{filename}.cage.npy', temp)